- Installazione dipendenze

In [ ]:
!pip install torch

!pip install datasets
!pip install tokenizers
!pip install transformers

!pip install seqeval

In [3]:
import os
import math
import random
import csv
import sys
import datasets
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
import statistics as stats

- lista labels

In [4]:
label_list=["O","B-Task","I-Task","B-Method","I-Method","B-Metric","I-Metric","B-Material","I-Material","B-OtherScientificTerm","I-OtherScientificTerm","B-Generic","I-Generic"]


label_map = {label: i for i, label in enumerate(label_list)}

- Caricamento predictions

In [ ]:
file_risultati="/content/drive/MyDrive/Tesi/dataset_nuovi/Model/result/result_predictions_without_inc.json"
y_pred_2 = datasets.load_dataset('json', data_files=file_risultati, split='train')

- caricamento dataset tokenizzati (ground truth)

In [6]:
tokenized_datasets=datasets.load_from_disk("/content/drive/MyDrive/Tesi/dataset_nuovi/Model/dataset_tokenizzati")

- Inizializzazione strutture dati per la valutazione

In [50]:
y_gt=[]
y_gt_WORDS=[]


for i in tokenized_datasets["test"]["labels"] :
    i=[x for x in i if x!=-100]
    #print(i)
    y_gt.extend(i)
    y_gt_WORDS.extend([ label_list[x] for x in i ])

y_pred=[]
y_pred_WORDS=[]

for i in y_pred_2['predictions_id'][0] :
  y_pred.extend(i)
  y_pred_WORDS.extend([ label_list[x] for x in i ])

- token by token

In [ ]:
accuracy = accuracy_score(y_gt_WORDS, y_pred_WORDS)
precision = precision_score(y_gt_WORDS, y_pred_WORDS, average='macro')
recall = recall_score(y_gt_WORDS, y_pred_WORDS, average='macro')
f1 = f1_score(y_gt_WORDS, y_pred_WORDS, average='macro')

print("Accuracy: ", accuracy)
print("Precision macro: ", precision)
print("Recall macro: ", recall)
print("F1 macro: ", f1)
print(classification_report(y_gt_WORDS,y_pred_WORDS,labels=label_list))



- SPAN

- Funzione che prende la predizione di una frase, ne estrae gli span e li salva in un dizionario in cui le chiavi sono le label e i valori i relativi span

In [7]:
def extract_span (lista_labels):


  

  B_types=["B-Task","B-Method","B-Metric","B-Material","B-OtherScientificTerm","B-Generic","O"]

  dict_span={}


  for t in B_types:



    end=-1
    if(t!="O"):
      occorrenze=[i for i,val in enumerate(lista_labels) if val[2:]==t[2:]]
      dict_span[t[2:]]=[]
      for x in occorrenze :
        start=x

        if((start+1 <= len(lista_labels)-1) and (start>end )):
          end=start
          while (lista_labels[end+1][2:]==t[2:]):
            end=end+1

            if (end+1 >= len(lista_labels)-1):
              break

          dict_span[t[2:]].append((start,end))
        else:
          ends=[y for x,y in dict_span[t[2:]]]
          if(ends==[]):
            dict_span[t[2:]].append((start,start))

          elif(start > ends[-1]):
            dict_span[t[2:]].append((start,start))
    else:
      occorrenze=[i for i,val in enumerate(lista_labels) if val==t]
      dict_span[t]=[]
      for x in occorrenze :
        start=x

        if((start+1 < len(lista_labels)-1)and (start>end)):
          end=start

          while ((end+1 <= len(lista_labels)-1)and lista_labels[end+1]=="O" ):
            end=end+1

            if (end+1 >= len(lista_labels)-1):
              break

          dict_span[t].append((start,end))
        else:
          ends=[y for x,y in dict_span[t]]
          if (start > ends[-1]):
            dict_span[t].append((start,start))



  return dict_span

- La funzione viene estesa a tutte le frasi per creare un dizionario completo di tutti gli span, sia per le predizioni che per la ground_truth

In [8]:
dict_pred={}
for i,j in enumerate(y_pred_2['predictions_id'][0]):
  j=[label_list[x] for x in j]
  dict_pred[i]=extract_span(j)

In [ ]:
dict_truth={}

for i,j in enumerate(tokenized_datasets["test"]["labels"]) :#label_list[x] for x
    j=[label_list[x] for x in j if x!=-100]
    dict_truth[i]=extract_span(j)


- Calcolo delle metriche su Span senza tipo

In [ ]:
true_positives = 0
false_positives = 0
false_negatives = 0
len_tot=0

for i in range(0,551):
  ground_truth=dict_truth[i]
  predicted=dict_pred[i]
  len_tot+=len(predicted.items())+len(ground_truth.items())

  entity_types = set(ground_truth.keys()).union(predicted.keys())
  gt_list = []
  prediction_list = []
  merged_list = []

  for entity_type in entity_types:
    if(entity_type in ground_truth.keys()):
      for element in ground_truth[entity_type]:
        gt_list.append(element)
        if(element not in merged_list):
          merged_list.append(element)
    if(entity_type in predicted.keys()):
      for element in predicted[entity_type]:
        prediction_list.append(element)
        if(element not in merged_list):
          merged_list.append(element)

  for element in merged_list:
      if(element in gt_list and element in prediction_list):
        true_positives += 1

      if(element not in gt_list and element in prediction_list):
        false_positives += 1

      if(element in gt_list and element not in prediction_list):
          false_negatives += 1


precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)
accuracy = true_positives / (true_positives + false_positives + false_negatives)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("Accuracy:",accuracy)


- Calcolo delle metriche su Span con tipo

In [ ]:
true_positives = 0
false_positives = 0
false_negatives = 0
len_tot=0
for i in range(0,551):
  ground_truth=dict_truth[i]
  predicted=dict_pred[i]
  len_tot+=len(predicted.items())+len(ground_truth.items())
  entity_types = set(ground_truth.keys()).union(predicted.keys())

  for entity_type in entity_types:
      # Get true positives
      true_positives += sum(1 for entity in ground_truth.get(entity_type, [])
                            if entity in predicted.get(entity_type, []))

      # Get false positives
      false_positives += sum(1 for entity in predicted.get(entity_type, [])
                              if entity not in ground_truth.get(entity_type, []))

      # Get false negatives
      false_negatives += sum(1 for entity in ground_truth.get(entity_type, [])
                              if entity not in predicted.get(entity_type, []))


precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)
accuracy = true_positives / (true_positives + false_positives + false_negatives)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("Accuracy:",accuracy)


- Salvataggio di entrambi i dizionari

In [ ]:
file_dict_span="/content/drive/MyDrive/Tesi/Model/result/span_dict_with_inc.json"


with open(file_dict_span, 'a') as f:
    json.dump({"predicted_dict":dict_pred,"ground_truth_dict":dict_truth}, f)

